In [1]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy_utils import create_database, database_exists, drop_database

## Extract CSVs into DataFrames

In [2]:
disaster_data = "../Resources/DisasterDeclarationsSummaries.csv"
disaster_data = pd.read_csv(disaster_data)
disaster_data.head()

,Declaration Number,Individuals and Households Program,Individual Assistance Program,Public Assistance Program,Hazard Mitigation Program,State,Pres_Id,Declaration Date,Declaration Type,Disaster Type,Disaster Title,Start Date,End Date,Closed Date,Place Code,County,Declaration Request Number
0,4419,Yes,No,Yes,Yes,AL,45,3/5/2019,DR,Tornado,"SEVERE STORMS, STRAIGHT-LINE WINDS, AND TORNADOES",3/3/2019,3/3/2019,Undetermined,99081,Lee (County),19006
1,4418,No,No,Yes,Yes,WA,45,3/4/2019,DR,Severe Storm(s),"SEVERE WINTER STORMS, STRAIGHT-LINE WINDS, FLO...",12/10/2018,12/24/2018,Undetermined,99009,Clallam (County),19005
2,4418,No,No,Yes,Yes,WA,45,3/4/2019,DR,Severe Storm(s),"SEVERE WINTER STORMS, STRAIGHT-LINE WINDS, FLO...",12/10/2018,12/24/2018,Undetermined,99027,Grays Harbor (County),19005
3,4418,No,No,Yes,Yes,WA,45,3/4/2019,DR,Severe Storm(s),"SEVERE WINTER STORMS, STRAIGHT-LINE WINDS, FLO...",12/10/2018,12/24/2018,Undetermined,99029,Island (County),19005
4,4418,No,No,Yes,Yes,WA,45,3/4/2019,DR,Severe Storm(s),"SEVERE WINTER STORMS, STRAIGHT-LINE WINDS, FLO...",12/10/2018,12/24/2018,Undetermined,99031,Jefferson (County),19005


In [3]:
presidents_data = "../Resources/Presidents.csv"
presidents_data = pd.read_csv(presidents_data)
presidents_data.head()

,Pres_Id,President,Term Start,Term End
0,1,George Washington,04/30/1789,03/04/1797
1,2,John Adams,03/04/1797,03/04/1801
2,3,Thomas Jefferson,03/04/1801,03/04/1809
3,4,James Madison,03/04/1809,03/04/1817
4,5,James Monroe,03/04/1817,03/04/1825


## Transform DataFrame

In [4]:
# Merge dataframes then sort by Pres_Id
disasters_declared = disaster_data.merge(presidents_data, on="Pres_Id", how="left")
disasters_declared.head ()

,Declaration Number,Individuals and Households Program,Individual Assistance Program,Public Assistance Program,Hazard Mitigation Program,State,Pres_Id,Declaration Date,Declaration Type,Disaster Type,Disaster Title,Start Date,End Date,Closed Date,Place Code,County,Declaration Request Number,President,Term Start,Term End
0,4419,Yes,No,Yes,Yes,AL,45,3/5/2019,DR,Tornado,"SEVERE STORMS, STRAIGHT-LINE WINDS, AND TORNADOES",3/3/2019,3/3/2019,Undetermined,99081,Lee (County),19006,Donald Trump,1/20/2017,6/7/2019
1,4418,No,No,Yes,Yes,WA,45,3/4/2019,DR,Severe Storm(s),"SEVERE WINTER STORMS, STRAIGHT-LINE WINDS, FLO...",12/10/2018,12/24/2018,Undetermined,99009,Clallam (County),19005,Donald Trump,1/20/2017,6/7/2019
2,4418,No,No,Yes,Yes,WA,45,3/4/2019,DR,Severe Storm(s),"SEVERE WINTER STORMS, STRAIGHT-LINE WINDS, FLO...",12/10/2018,12/24/2018,Undetermined,99027,Grays Harbor (County),19005,Donald Trump,1/20/2017,6/7/2019
3,4418,No,No,Yes,Yes,WA,45,3/4/2019,DR,Severe Storm(s),"SEVERE WINTER STORMS, STRAIGHT-LINE WINDS, FLO...",12/10/2018,12/24/2018,Undetermined,99029,Island (County),19005,Donald Trump,1/20/2017,6/7/2019
4,4418,No,No,Yes,Yes,WA,45,3/4/2019,DR,Severe Storm(s),"SEVERE WINTER STORMS, STRAIGHT-LINE WINDS, FLO...",12/10/2018,12/24/2018,Undetermined,99031,Jefferson (County),19005,Donald Trump,1/20/2017,6/7/2019


In [5]:
# Sort data by Declaration Number
disasters_transformed=disasters_declared.sort_values("Declaration Number")
disasters_transformed.head()

,Declaration Number,Individuals and Households Program,Individual Assistance Program,Public Assistance Program,Hazard Mitigation Program,State,Pres_Id,Declaration Date,Declaration Type,Disaster Type,Disaster Title,Start Date,End Date,Closed Date,Place Code,County,Declaration Request Number,President,Term Start,Term End
48554,1,No,Yes,Yes,Yes,GA,34,5/2/1953,DR,Tornado,TORNADO,5/2/1953,5/2/1953,6/1/1954,Undetermined,Undetermined,53013,Dwight Eisenhower,1/20/1953,1/20/1961
48553,2,No,Yes,Yes,Yes,TX,34,5/15/1953,DR,Tornado,TORNADO & HEAVY RAINFALL,5/15/1953,5/15/1953,1/1/1958,Undetermined,Undetermined,53003,Dwight Eisenhower,1/20/1953,1/20/1961
48552,3,No,Yes,Yes,Yes,LA,34,5/29/1953,DR,Flood,FLOOD,5/29/1953,5/29/1953,2/1/1960,Undetermined,Undetermined,53005,Dwight Eisenhower,1/20/1953,1/20/1961
48551,4,No,Yes,Yes,Yes,MI,34,6/2/1953,DR,Tornado,TORNADO,6/2/1953,6/2/1953,2/1/1956,Undetermined,Undetermined,53004,Dwight Eisenhower,1/20/1953,1/20/1961
48550,5,No,Yes,Yes,Yes,MT,34,6/6/1953,DR,Flood,FLOODS,6/6/1953,6/6/1953,12/1/1955,Undetermined,Undetermined,53006,Dwight Eisenhower,1/20/1953,1/20/1961


In [6]:
disasters_transformed.columns

Index(['Declaration Number', 'Individuals and Households Program',
       'Individual Assistance Program', 'Public Assistance Program',
       'Hazard Mitigation Program', 'State ', 'Pres_Id', 'Declaration Date',
       'Declaration Type', 'Disaster Type', 'Disaster Title', 'Start Date',
       'End Date', 'Closed Date', 'Place Code', 'County',
       'Declaration Request Number', 'President', 'Term Start', 'Term End'],
      dtype='object')

In [7]:
#Rearrange columns in meaningful order 
disasters_finaltrans=disasters_transformed[['Pres_Id','President','State ','County','Declaration Date','Declaration Number','Declaration Type', 'Disaster Type', 'Disaster Title', 'Start Date',
       'End Date', 'Closed Date','Individuals and Households Program',
       'Individual Assistance Program', 'Public Assistance Program',
       'Hazard Mitigation Program']]
disasters_finaltrans.head()

,Pres_Id,President,State,County,Declaration Date,Declaration Number,Declaration Type,Disaster Type,Disaster Title,Start Date,End Date,Closed Date,Individuals and Households Program,Individual Assistance Program,Public Assistance Program,Hazard Mitigation Program
48554,34,Dwight Eisenhower,GA,Undetermined,5/2/1953,1,DR,Tornado,TORNADO,5/2/1953,5/2/1953,6/1/1954,No,Yes,Yes,Yes
48553,34,Dwight Eisenhower,TX,Undetermined,5/15/1953,2,DR,Tornado,TORNADO & HEAVY RAINFALL,5/15/1953,5/15/1953,1/1/1958,No,Yes,Yes,Yes
48552,34,Dwight Eisenhower,LA,Undetermined,5/29/1953,3,DR,Flood,FLOOD,5/29/1953,5/29/1953,2/1/1960,No,Yes,Yes,Yes
48551,34,Dwight Eisenhower,MI,Undetermined,6/2/1953,4,DR,Tornado,TORNADO,6/2/1953,6/2/1953,2/1/1956,No,Yes,Yes,Yes
48550,34,Dwight Eisenhower,MT,Undetermined,6/6/1953,5,DR,Flood,FLOODS,6/6/1953,6/6/1953,12/1/1955,No,Yes,Yes,Yes


In [8]:
Disasters_byPres=disasters_finaltrans.groupby("President")["Declaration Date"].count().reset_index()

# Rename the column headers
Disasters_byPres=Disasters_byPres.rename(columns={"Declaration Date": "Count of Disasters Declared"})
Disasters_byPres

,President,Count of Disasters Declared
0,Barack Obama,10250
1,Bill Clinton,9697
2,Donald Trump,2480
3,Dwight Eisenhower,106
4,George Bush,2083
5,George W. Bush,15107
6,Gerald Ford,1183
7,Jimmy Carter,2261
8,John F. Kennedy,52
9,Lyndon B. Johnson,1046


In [9]:
disasters_finaltrans.to_csv('C:\\Users\\monic\\Desktop\\ETL\\new_disasterfile.csv')

## Create Database Connection

In [10]:
connection_string = "postgres:Raffy1768@localhost:5432/disasters_db"
engine = create_engine(f'postgresql://{connection_string}')
engine.table_names()

['disasters']

In [11]:
connection = engine.connect()
Base = declarative_base()
Base.metadata.drop_all(engine)

In [12]:
# Confirm tables
engine.table_names()


['disasters']

## Load DataFrames into database

In [13]:
disasters_finaltrans.to_sql(name='disasters', con=engine, if_exists='append', index=True)